In [6]:
import os, csv, re

First we create an empty CSV file with the colums to fill.

In [3]:
def create_empty_csv(filename):
    csvfile = open('data/output/{0}.csv'.format(filename), "w")
    writer = csv.writer(csvfile)
    columns = ['filename', 'title', 'author', 'release year', 'pos score', 'neg score', 'neu score', 'comp score']
    writer.writerow(columns)

create_empty_csv('tard')


We want to be able to extract the filename, title, author and release year from every book and save it to a CSV file. 

In [7]:
def obtain_data_from_books(directory):
    
    # initiate CSV file 
    csvfile = open('data/output/tard.csv', "a")
    writer = csv.writer(csvfile)
  
    for book in os.listdir(directory):
        if not book.startswith("."):
            with open(directory + book, errors='replace') as f:
                text = f.read().splitlines()
        
        author = "NULL"
        title = "NULL"
        release_date = "NULL"
        
        # for line in text, check if title, author or release date are stored there
        for i in range(80):
            # error handling since some texts are <80 lines
            try:
                if "Title: " in text[i]:
                    title = text[i][7:]
                if "Author: " in text[i]:
                    author = text[i][8:]
                if "Release Date: " in text[i]:
                    release_date = re.findall('(\d{4})', text[i][14:].split('[')[0])
                # if they have both been found, do not waste extra time iterating 
                if title != "NULL" and author != "NULL" and release_date != "NULL":
                    title_author_rd_triple = (title, author, release_date)
                    pass
            except:
                pass
            
        data = [book, title, author, release_date]
        if not 'NULL' in data:
            writer.writerow(data)
            print (data)

    
directory = "/Users/Tristan/Downloads/9copy/Jesus/"
obtain_data_from_books(directory)


['9396.txt', 'Atlantic Monthly, Vol. 5, No. 30, April, 1860', 'Various', ['2005']]
['9382.txt', "Grandma's Memories", 'Mary D. Brine', ['2004']]
['9633-8.txt', 'Sir George Tressady, Vol. I', 'Mrs. Humphry Ward', ['2006']]
['909.txt', 'The Adventures of Colonel Daniel Boone', 'John Filson', ['1997']]
['9619.txt', 'Who Can Be Happy And Free In Russia?', 'Nicholas Nekrassov', ['2006']]
['9497-8.txt', 'Twilight in Italy', 'D. H. Lawrence', ['2005']]
['921.txt', 'De Profundis', 'Oscar Wilde', ['2007']]
['9631.txt', 'Waltonia', 'Isaak Walton', ['2006']]
['9625.txt', 'Buried Cities, Part 1, Pompeii', 'Jennie Hall', ['2004']]
['9194.txt', 'The Second Deluge', 'Garrett P. Serviss', ['2005']]
['9412-8.txt', 'A Woman Tenderfoot', 'Grace Gallatin Seton-Thompson', ['2005']]
['9745.txt', 'The Rock of Chickamauga', 'Joseph A. Altsheler', ['2006']]
['9751.txt', 'Night and Morning, Volume 2', 'Edward Bulwer Lytton', ['2006']]
['9387-8.txt', 'Theresa Marchmont', 'Mrs Charles Gore', ['2005']]
['9209.txt'

['942.txt', 'Green Mansions', 'W. H. Hudson', ['1997']]
['9470-8.txt', 'His Hour', 'Elinor Glyn', ['2005']]
['9652.txt', 'A Miscellany of Poetry', 'Various', ['2006']]
['956.txt', 'Tik-Tok of Oz', 'L. Frank Baum', ['1997']]
['981.txt', 'Beowulf', 'Anonymous', ['1997']]
['995.txt', 'Ballads of a Bohemian', 'Robert W. Service', ['1997']]
['9061-8.txt', 'The Public Orations of Demosthenes, volume 2', 'Demosthenes', ['2005']]
['9068.txt', 'Apu Ollantay', 'Anonymous', ['2005']]
['9054.txt', "The Young Woman's Guide", 'William A. Alcott', ['2005']]
['9097.txt', "Robert's Rules of Order", 'Henry M. Robert', ['2004']]
['9242.txt', 'Old Ticonderoga, A Picture of The Past', 'Nathaniel Hawthorne', ['2005']]
['9256.txt', 'The Paradise of Children', 'Nathaniel Hawthorne', ['2005']]
['9614-8.txt', 'The Case of Richard Meynell', 'Mrs. Humphrey Ward', ['2006']]
['9295.txt', 'A Night Out', 'Edward Peple', ['2005']]
['9651-8.txt', 'A Child-World', 'James Whitcomb Riley', ['2006']]
['9469-8.txt', 'The Ca

['9069-8.txt', 'Saint Augustin', 'Louis Bertrand', ['2005']]
['9061.txt', 'The Public Orations of Demosthenes, volume 2', 'Demosthenes', ['2005']]
['9485-8.txt', 'A Little Book of Profitable Tales', 'Eugene Field', ['2005']]
['9054-8.txt', "The Young Woman's Guide", 'William A. Alcott', ['2005']]
['9075.txt', 'Rico and Wiseli', 'Johanna Spyri', ['2005']]
['9310-8.txt', "Casanova's Homecoming", 'Arthur Schnitzler', ['2005']]
['9263.txt', 'In the Midst of Alarms', 'Robert Barr', ['2005']]
['9505.txt', 'Four Girls and a Compact', 'Annie Hamilton Donnell', ['2005']]
['9664-8.txt', 'An Amiable Charlatan', 'E. Phillips Oppenheim', ['2006']]
['9621-8.txt', 'Georgian Poetry 1918-19', 'Various', ['2006']]
['9149-8.txt', 'The Gray Dawn', 'Stewart Edward White', ['2005']]
['9334-8.txt', 'The Spectator, Volume 1', 'Joseph Addison and Richard Steele', ['2005']]
['9371-8.txt', 'The Praise of Folly', 'Desiderius Erasmus', ['2005']]
['9070-8.txt', 'The Imaginary Invalid', 'Moli�re', ['2003']]
['9504.t

['9073-8.txt', 'The Roof of France', 'Matilda Betham-Edwards', ['2005']]
['9610.txt', 'The Elegies of Tibullus', 'Tibullus', ['2006']]
['9604.txt', 'Hung Lou Meng, Book II', 'Cao Xueqin', ['2006']]
['9162.txt', 'Becket and other plays', 'Alfred Lord Tennyson', ['2005']]
['9189.txt', 'Henry Dunbar', 'M. E. Braddon', ['2005']]
['9372-8.txt', 'Songs Out of Doors', 'Henry Van Dyke', ['2005']]
['9758.txt', 'Leila or, The Siege of Granada, Book III.', 'Edward Bulwer Lytton', ['2006']]
['9764.txt', 'Alice, or The Mysteries, Book II', 'Edward Bulwer Lytton', ['2006']]
['9228.txt', 'The Christmas Banquet (From "Mosses From An Old Manse")', 'Nathaniel Hawthorne', ['2005']]
['940-8.txt', 'The Last of the Mohicans', 'James Fenimore Cooper', ['2006']]
['9214.txt', 'The White Old Maid (From "Twice Told Tales")', 'Nathaniel Hawthorne', ['2005']]
['9599.txt', 'The Works of Whittier, Volume VII (of VII)', 'John Greenleaf Whittier', ['2005']]
['9396-8.txt', 'Atlantic Monthly, Vol. 5, No. 30, April, 1860

['9601.txt', 'Poetical Works of Pope, Vol. II', 'Alexander Pope', ['2006']]
['9629.txt', 'Ghost Stories of an Antiquary', 'Montague Rhodes James', ['2006']]
['9442-8.txt', 'Helbeck of Bannisdale, Vol. II', 'Mrs. Humphry Ward', ['2005']]
['9198.txt', 'The Complete Angler, 1653', 'Isaak Walton', ['2005']]
['9761.txt', 'Leila, Complete', 'Edward Bulwer-Lytton', ['2009']]
['9482-8.txt', 'What Sami Sings with the Birds', 'Johanna Spyri', ['2005']]
['9749.txt', 'The Highwayman', 'H. C. Bailey', ['2006']]
['9205.txt', 'Sights from a Steeple (From "Twice Told Tales")', 'Nathaniel Hawthorne', ['2005']]
['9211.txt', 'The Sister Years (From "Twice Told Tales")', 'Nathaniel Hawthorne', ['2005']]
['9239.txt', 'Old News', 'Nathaniel Hawthorne', ['2005']]
['9663-8.txt', 'Domnei', 'James Branch Cabell', ['2006']]
['9376-8.txt', 'The Forest', 'Stewart Edward White', ['2005']]
['9333-8.txt', 'Johnny Bear', 'E. T. Seton', ['2005']]
['9238.txt', 'Sylph Etherege', 'Nathaniel Hawthorne', ['2005']]
['9210.tx

Delete header and legal stuff at the beginning and end of the books. Books from the gutenberg project have certain sentences that mark the start and the end of a book. This can be exploited to extract only the content from the books that is relevant to our research. The function below does this for all books in the given directory and puts the stripped books in a new directory. The unstripped books are kept because some useful information is found in the headers (author, title, release date)

In [9]:
def remove_unwanted_text(directory, outputdir):

    # lists with sentences that either mark the end or the beginning of a book
    start_list = ("*** START OF", "***START OF")
    stop_list = ("*** END OF", "***END OF", "End of the Project")

    for book in os.listdir(directory):
        print(book)
        if not book.startswith("."):
            with open(directory + book, errors='replace') as f:
                content = f.readlines()
                
                start_index = 1
                stop_index = len(content) - 1

                for line in range(len(content)):

                    if any(item in content[line] for item in start_list):
                        start_index = line + 1
                    #if stop1 or stop2 or stop3 in content[line]:
                    if any(item in content[line] for item in stop_list):
                        stop_index = line - 2

                book_content = content[start_index:stop_index]
                outfile = open(outputdir + '/' + book, 'w')
                outfile.writelines(book_content)
                outfile.close()
        

directory = "data/test/"
outputdir = "data/output/"
remove_unwanted_text(directory, outputdir)
   

.DS_Store
